In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import re

def read_map_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    resolution = float(lines[0].split(': ')[1])
    width = int(lines[1].split(': ')[1])
    height = int(lines[2].split(': ')[1])

    origin_x = float(lines[4].split(': ')[1])
    origin_y = float(lines[5].split(': ')[1])

    data_start_index = lines.index('Data:\n') + 1
    data = []
    for line in lines[data_start_index:]:
        data.extend([int(value) for value in line.split()])

    data = np.array(data, dtype=np.float32).reshape((height, width))
    return resolution, width, height, data, {'x': origin_x, 'y': origin_y}


def read_gassource_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if 'translation:' in line:
            x = float(lines[lines.index(line) + 1].split(': ')[1])
            y = float(lines[lines.index(line) + 2].split(': ')[1])
            break

    return x, y

def f(x):#loss function threshold
    a = 3
    b = 0.3
    c = 1
    return a * np.exp(-b * (x-1)) + c


def convert_coordinates(x, y, resolution, origin):
    x_map = (x - origin['x']) / resolution
    y_map = (y - origin['y']) / resolution
    return x_map, y_map


def process_map_file(map_file, gas_x, gas_y, output_file, D):
    resolution, width, height, data, origin = read_map_file(map_file)

    # 将气源坐标转换为地图坐标
    gas_x_map, gas_y_map = convert_coordinates(gas_x, gas_y, resolution, origin)

    d=f(D)
    for y in range(height):
        for x in range(width):
            distance = np.sqrt((x - gas_x_map) ** 2 + (y - gas_y_map) ** 2) * resolution

            if distance > d or data[y, x]== -1:
                data[y, x] = 0.0
            else:
                #print(f"x: {x}, y: {y}, distance: {distance}")
                o=data[y, x]
                data[y, x] = ((d-distance)/d)*(100-o)/100
                #print(data[y, x])

    write_map_file(output_file, resolution, width, height, data, origin)


def write_map_file(file_path, resolution, width, height, data, origin):
    with open(file_path, 'w') as file:
        file.write(f"Resolution: {resolution}\n")
        file.write(f"Width: {width}\n")
        file.write(f"Height: {height}\n")
        file.write("Origin: position: \n")
        file.write(f"  x: {origin['x']}\n")
        file.write(f"  y: {origin['y']}\n")
        file.write("  z: 0.0\n")
        file.write("orientation: \n")
        file.write("  x: 0.0\n")
        file.write("  y: 0.0\n")
        file.write("  z: 0.0\n")
        file.write("  w: 1.0\n")
        file.write("Data:\n")

        for row in data:
            file.write(" ".join(f"{value:.2f}" for value in row) + "\n")


def visualize_label_map(file_path,save=True):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    width = int(lines[1].split(': ')[1])
    height = int(lines[2].split(': ')[1])

    data_start_index = lines.index('Data:\n') + 1
    data = []
    for line in lines[data_start_index:]:
        data.extend([float(value) for value in line.split()])

    data = np.array(data).reshape((height, width))

    plt.figure(figsize=(8, 8))
    plt.imshow(data, cmap='Reds', origin='lower')
    plt.colorbar(label='Labeled Value')
    #plt.title(f'Visualization of {file_path}')
    if save:
        output_path = file_path.replace('.txt', '_visualization.png')
        plt.savefig(output_path, dpi=300)
    #plt.show()
    plt.close()


def main(epoch_num):
    base_folder_path = r'/content/drive/MyDrive/OSLAM_result'
    folder_name = f'epoch{epoch_num}'
    folder_path = os.path.join(base_folder_path, folder_name)


    gassource_file = os.path.join(folder_path, 'gassource_baselink_to_map.txt')
    gas_x, gas_y = read_gassource_file(gassource_file)

    map_files = sorted([f for f in os.listdir(folder_path) if re.match(r'^map\d+\.txt$', f)])

    for map_file in map_files:
        index = map_file.split('map')[-1].split('.')[0]
        map_file_path = os.path.join(folder_path, map_file)
        output_file_path = os.path.join(folder_path, f'label{index}.txt')
        process_map_file(map_file_path, gas_x, gas_y, output_file_path,float(index))

        visualize_label_map(output_file_path)

def vis_result(epoch_num):
    base_folder_path = r'/content/drive/MyDrive/OSLAM_result'
    folder_name = f'epoch{epoch_num}'
    folder_path = os.path.join(base_folder_path, folder_name)

    result_files = sorted([f for f in os.listdir(folder_path) if re.match(r'^label\d+\.txt$', f)])

    for result in result_files:
        index = result.split('map')[-1].split('.')[0]
        result_file_path = os.path.join(folder_path, result)
        #result_path = os.path.join(result_file_path, result)
        visualize_label_map(result_file_path)

if __name__ == "__main__":
    for i in range(1,65):
        main(i)
        #vis_result(i)
